In [98]:
import re
import os
import pandas as pd

## Instruções

1. **Pasta de Entrada (`base`)**:
   - Utilize a pasta `base` para armazenar o arquivo que contém os CPFs ou CNPJs a serem tratados e validados.
   - O código identificará e processará automaticamente o **primeiro arquivo encontrado** nesta pasta.

2. **Configuração do Nome da Coluna (`coluna_verificadora`)**:
   - Especifique o nome da coluna que contém os dados a serem tratados (por exemplo: `cpf_cnpj`).

3. **Configurações para Arquivos `.csv`**:
   - Ajuste os parâmetros abaixo, se necessário:
     - **`csv_delimiter`**: Define o delimitador utilizado no arquivo (padrão: `;`).
     - **`csv_encoding`**: Define a codificação do arquivo (padrão: `utf-8`).

4. **Configurações para Arquivos `.xlsx`**:
   - Nenhuma configuração adicional é necessária para arquivos do tipo `.xlsx`.

5. **Exportação dos Resultados (`verif_export`)**:
   - Quando o parâmetro **`verif_export`** estiver configurado como `True`, o arquivo tratado será salvo na pasta `export`.
   - O arquivo final será automaticamente nomeado e salvo como `output.xlsx`.

## Configurações iniciais

In [99]:

formato_selec = "Padrao"                # SACL ou Padrao
coluna_verificadora = "cpf_cnpj"        # Nome da coluna do documento
verif_export = True                     # Deseja exportar ao finalizar?
caminho_output = r'export\output.xlsx'  # Não precisa alterar
diretorio_base = r'base'                # Não precisa alterar
csv_delimiter = ";"                     # Delimitador do CSV
csv_encoding = "utf-8"                  # Codificação do CSV

## Dados

In [100]:
# primeiro arquivo excel ou csv na pasta
arquivos = [f for f in os.listdir(diretorio_base) if f.endswith(('.xls', '.xlsx', '.csv'))]

if arquivos:
    caminho_arquivo = os.path.join(diretorio_base, arquivos[0])
    if caminho_arquivo.endswith(('.xls', '.xlsx')):
        data = pd.read_excel(caminho_arquivo, usecols=[coluna_verificadora])
    elif caminho_arquivo.endswith('.csv'):
        data = pd.read_csv(
            caminho_arquivo,
            usecols=[coluna_verificadora],
            delimiter=csv_delimiter,
            encoding=csv_encoding
        )
    print(f"Arquivo carregado: {caminho_arquivo}")
else:
    raise FileNotFoundError(f"Nenhum arquivo Excel ou CSV encontrado na pasta \{diretorio_base}.")

formato_selec = formato_selec.lower().strip()
data.head()

Arquivo carregado: base\base_cpf_cnpj.xlsx


,cpf_cnpj
0,22536471284
1,41209745372
2,41512145343
3,64287054270
4,69346060000167


In [101]:
data.dtypes

cpf_cnpj    int64
dtype: object

## Normalização
- Caso a quantidade de dígitos for menor que 11, é adicionado zeros à esquerda até completar 11 dígitos
- Caso a quantidade de dígitos for igual a 13, é adicionado zeros à esquerda até completar 14 dígitos
- Isso reduz a taxa de erro em bases poluídas, porém ainda há a possibilidade de ter sequencias inválidas

In [102]:
def tratamento(documento):
    documento = re.sub(r'\D', '', documento) # remove caracteres não numéricos
    documento = documento.replace(" ", "") # remove espaços em branco

    documento = str(documento)
    
    # Adiciona zeros à esquerda
    if len(documento) < 11:
        return documento.zfill(11)
    elif len(documento) > 11 and len(documento) < 14:
        return documento.zfill(14)
    
    return documento

In [103]:
data[coluna_verificadora] = data[coluna_verificadora].astype(str).apply(tratamento)

# Coluna com a quantidade de caracteres pós tratamento
data['tam_inicial'] = data[coluna_verificadora].apply(len)
data.head()

,cpf_cnpj,tam_inicial
0,22536471284,11
1,41209745372,11
2,41512145343,11
3,64287054270,11
4,69346060000167,14


## Identificador CPF/CNPJ

In [104]:
# 0001 indica a matriz do estabelecimento, sendo o mais comum
# do 0002 ao 9999 indica filiais
# considerei apenas do 0001 ao 0009 para evitar conflito na hora da verificação e por ser os mais comuns
# caso necessário, adicionar mais
filial_cnpj = ['0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009']

def identificador(documento, formato):
    
    documento = re.sub(r'\D', '', documento)
    
    # Padrão (11 ou 14 dígitos)
    if formato == "padrao":
        if documento[-6:-2] in filial_cnpj:
            return "CNPJ"
        elif len(documento) == 14:
            return "CNPJ"
        elif len(documento) == 11:
            return "CPF"
        else:
            return "N/A"
        
    # SACL (15 dígitos)
    elif formato == "sacl":
        if len(documento) == 15:
            if documento[-6:-2] == "0000":
                return "CPF"
            else:
                return "CNPJ"
            
    return "N/A"

In [105]:
data['verificador'] = data[coluna_verificadora].apply(lambda x: identificador(x, formato_selec))
data.head()

,cpf_cnpj,tam_inicial,verificador
0,22536471284,11,CPF
1,41209745372,11,CPF
2,41512145343,11,CPF
3,64287054270,11,CPF
4,69346060000167,14,CNPJ


## Apenas números

In [106]:
def cpf_cnpj_numerico(row, formato_selec):

    cpf_cnpj = row[coluna_verificadora]
    verificador = row["verificador"]

    # CNPJ SACL 15 dígitos
    if verificador == "CNPJ" and formato_selec == "sacl" and len(cpf_cnpj) == 15:
        return cpf_cnpj[1:]

    # CPF SACL 15 dígitos
    elif verificador == "CPF" and formato_selec == "sacl" and len(cpf_cnpj) == 15:
        return cpf_cnpj[:9] + cpf_cnpj[-2:]

    # CNPJ Padrao <14 dígitos
    elif verificador == "CNPJ" and formato_selec == "padrao" and len(cpf_cnpj) < 14:
        return cpf_cnpj.zfill(14) 

    # CPF Padrao <11 dígitos
    elif verificador == "CPF" and formato_selec == "padrao" and len(cpf_cnpj) < 11:
        return cpf_cnpj.zfill(11)

    return cpf_cnpj

In [107]:
data["numerico"] = data.apply(lambda row: cpf_cnpj_numerico(row, formato_selec), axis=1)
data["tam_numerico"] = data['numerico'].apply(len)
data.head()

,cpf_cnpj,tam_inicial,verificador,numerico,tam_numerico
0,22536471284,11,CPF,22536471284,11
1,41209745372,11,CPF,41209745372,11
2,41512145343,11,CPF,41512145343,11
3,64287054270,11,CPF,64287054270,11
4,69346060000167,14,CNPJ,69346060000167,14


## Formatado com pontos, hífens e barras

In [108]:
def cpf_cnpj_formatado(row):

    cpf_cnpj = row["numerico"]
    verificador = row["verificador"]

    # CPF
    if verificador == "CPF" and len(cpf_cnpj) == 11:
        return f"{cpf_cnpj[:3]}.{cpf_cnpj[3:6]}.{cpf_cnpj[6:9]}-{cpf_cnpj[9:]}"

    # CNPJ
    elif verificador == "CNPJ" and len(cpf_cnpj) == 14:
        return f"{cpf_cnpj[:2]}.{cpf_cnpj[2:5]}.{cpf_cnpj[5:8]}/{cpf_cnpj[8:12]}-{cpf_cnpj[12:]}"
    
    return cpf_cnpj

In [109]:
data["formatado"] = data.apply(cpf_cnpj_formatado, axis=1)
data.head()

,cpf_cnpj,tam_inicial,verificador,numerico,tam_numerico,formatado
0,22536471284,11,CPF,22536471284,11,225.364.712-84
1,41209745372,11,CPF,41209745372,11,412.097.453-72
2,41512145343,11,CPF,41512145343,11,415.121.453-43
3,64287054270,11,CPF,64287054270,11,642.870.542-70
4,69346060000167,14,CNPJ,69346060000167,14,69.346.060/0001-67


## Validação dos dígitos verificadores

In [110]:
def validar_digitos_cpf(cpf):
    cpf = [int(d) for d in cpf]  # converte cada dígito em um inteiro

    # primeiro dígito
    soma1 = sum(cpf[i] * (10 - i) for i in range(9))
    dv1 = (soma1 * 10) % 11
    dv1 = 0 if dv1 == 10 else dv1

    # segundo dígito
    soma2 = sum(cpf[i] * (11 - i) for i in range(10))
    dv2 = (soma2 * 10) % 11
    dv2 = 0 if dv2 == 10 else dv2

    return dv1 == cpf[9], dv2 == cpf[10]

In [111]:
def validar_digitos_cnpj(cnpj):
    cnpj = [int(d) for d in cnpj]  # converte cada dígito em um inteiro

    # primeiro dígito
    pesos1 = [5, 4, 3, 2, 9, 8, 7, 6, 5, 4, 3, 2]
    soma1 = sum(cnpj[i] * pesos1[i] for i in range(12))
    dv1 = soma1 % 11
    dv1 = 0 if dv1 < 2 else 11 - dv1

    # segundo dígito
    pesos2 = [6] + pesos1
    soma2 = sum(cnpj[i] * pesos2[i] for i in range(13))
    dv2 = soma2 % 11
    dv2 = 0 if dv2 < 2 else 11 - dv2

    return dv1 == cnpj[12], dv2 == cnpj[13]

In [112]:
def extrair_e_verificar_dv(row):
    cpf_cnpj = row["numerico"]
    verificador = row["verificador"]
    dv1, dv2, verif_1, verif_2 = None, None, False, False
    if verificador == "CPF" and len(cpf_cnpj) == 11:
        dv1, dv2 = int(cpf_cnpj[9]), int(cpf_cnpj[10])
        verif_1, verif_2 = validar_digitos_cpf(cpf_cnpj)
    elif verificador == "CNPJ" and len(cpf_cnpj) == 14:
        dv1, dv2 = int(cpf_cnpj[12]), int(cpf_cnpj[13])
        verif_1, verif_2 = validar_digitos_cnpj(cpf_cnpj)
    return dv1, verif_1, dv2, verif_2

In [113]:
data["primeiro_dv"], data["verif_1"], data["segundo_dv"], data["verif_2"] = zip(*data.apply(extrair_e_verificar_dv, axis=1))
data.head()

,cpf_cnpj,tam_inicial,verificador,numerico,tam_numerico,formatado,primeiro_dv,verif_1,segundo_dv,verif_2
0,22536471284,11,CPF,22536471284,11,225.364.712-84,8,True,4,True
1,41209745372,11,CPF,41209745372,11,412.097.453-72,7,True,2,True
2,41512145343,11,CPF,41512145343,11,415.121.453-43,4,True,3,True
3,64287054270,11,CPF,64287054270,11,642.870.542-70,7,True,0,True
4,69346060000167,14,CNPJ,69346060000167,14,69.346.060/0001-67,6,True,7,True


In [114]:
def analisar_geografia(row):
    cpf_cnpj = row["numerico"]
    verificador = row["verificador"]

    if verificador == "CPF" and len(cpf_cnpj) >= 9:
        oitavo_digito = int(cpf_cnpj[8])  # nono dígito
        if oitavo_digito == 0:
            return "Rio Grande do Sul (RS)"
        elif oitavo_digito == 1:
            return "Centro-Oeste (DF, GO, MT, MS, TO)"
        elif oitavo_digito == 2:
            return "Região Norte (PA, AM, AC, AP, RO, RR)"
        elif oitavo_digito == 3:
            return "Ceará, Maranhão, Piauí (CE, MA, PI)"
        elif oitavo_digito == 4:
            return "Pernambuco, Rio Grande do Norte, Paraíba, Alagoas (PE, RN, PB, AL)"
        elif oitavo_digito == 5:
            return "Bahia, Sergipe (BA, SE)"
        elif oitavo_digito == 6:
            return "Minas Gerais (MG)"
        elif oitavo_digito == 7:
            return "Rio de Janeiro, Espírito Santo (RJ, ES)"
        elif oitavo_digito == 8:
            return "São Paulo (SP)"
        elif oitavo_digito == 9:
            return "Paraná, Santa Catarina, Rio Grande do Sul (PR, SC, RS)"
    return "N/A"

## Dataframe final

In [115]:
data["regiao_emissao"] = data.apply(analisar_geografia, axis=1)
data.head()

,cpf_cnpj,tam_inicial,verificador,numerico,tam_numerico,formatado,primeiro_dv,verif_1,segundo_dv,verif_2,regiao_emissao
0,22536471284,11,CPF,22536471284,11,225.364.712-84,8,True,4,True,"Região Norte (PA, AM, AC, AP, RO, RR)"
1,41209745372,11,CPF,41209745372,11,412.097.453-72,7,True,2,True,"Ceará, Maranhão, Piauí (CE, MA, PI)"
2,41512145343,11,CPF,41512145343,11,415.121.453-43,4,True,3,True,"Ceará, Maranhão, Piauí (CE, MA, PI)"
3,64287054270,11,CPF,64287054270,11,642.870.542-70,7,True,0,True,"Região Norte (PA, AM, AC, AP, RO, RR)"
4,69346060000167,14,CNPJ,69346060000167,14,69.346.060/0001-67,6,True,7,True,N/A


## Export

In [116]:
if verif_export == True:
    data.to_excel(caminho_output, index=False)

## Estatísticas

#### Caracteres iniciais

In [117]:
volumetria = data['tam_inicial'].value_counts()
percentual = data['tam_inicial'].value_counts(normalize=True) * 100
resultado = pd.DataFrame({
    'Volumetria': volumetria,
    'Percentual (%)': percentual
})

resultado = resultado.sort_values(by='Volumetria', ascending=False)
resultado

,Volumetria,Percentual (%)
tam_inicial,,
11,64,64.0
14,36,36.0


#### CPF/CNPJ

In [118]:
volumetria = data['verificador'].value_counts()
percentual = data['verificador'].value_counts(normalize=True) * 100
resultado = pd.DataFrame({
    'Volumetria': volumetria,
    'Percentual (%)': percentual
})

resultado = resultado.sort_values(by='Volumetria', ascending=False)
resultado

,Volumetria,Percentual (%)
verificador,,
CPF,64,64.0
CNPJ,36,36.0


#### Dígito 1

In [119]:
volumetria = data['verif_1'].value_counts()
percentual = data['verif_1'].value_counts(normalize=True) * 100
resultado = pd.DataFrame({
    'Volumetria': volumetria,
    'Percentual (%)': percentual
})

resultado = resultado.sort_values(by='Volumetria', ascending=False)
resultado

,Volumetria,Percentual (%)
verif_1,,
True,100,100.0


#### Dígito 2

In [120]:
volumetria = data['verif_2'].value_counts()
percentual = data['verif_2'].value_counts(normalize=True) * 100
resultado = pd.DataFrame({
    'Volumetria': volumetria,
    'Percentual (%)': percentual
})

resultado = resultado.sort_values(by='Volumetria', ascending=False)
resultado

,Volumetria,Percentual (%)
verif_2,,
True,100,100.0


#### Região de emissão

In [121]:
volumetria = data['regiao_emissao'].value_counts()
percentual = data['regiao_emissao'].value_counts(normalize=True) * 100
resultado = pd.DataFrame({
    'Volumetria': volumetria,
    'Percentual (%)': percentual
})

resultado = resultado.sort_values(by='Volumetria', ascending=False)
resultado

,Volumetria,Percentual (%)
regiao_emissao,,
N/A,36,36.0
"Ceará, Maranhão, Piauí (CE, MA, PI)",12,12.0
"Região Norte (PA, AM, AC, AP, RO, RR)",8,8.0
Rio Grande do Sul (RS),7,7.0
"Pernambuco, Rio Grande do Norte, Paraíba, Alagoas (PE, RN, PB, AL)",7,7.0
"Rio de Janeiro, Espírito Santo (RJ, ES)",6,6.0
"Paraná, Santa Catarina, Rio Grande do Sul (PR, SC, RS)",6,6.0
"Bahia, Sergipe (BA, SE)",6,6.0
Minas Gerais (MG),5,5.0
